In [9]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:
def h_acc(state, a_n, omega_n):
    R_imu, v_imu, p_imu, b_a, b_omega, R_c, p_c = state

    v_c = R_imu.T @ R_C.T @ v_imu + np.cross(omega_n - b_omega, p_c)
    obs = a_n[1] - v_c.flatten()[0] * (omega_n - b_omega)[3] 
    return obs

In [ ]:
R_imu = R.from_rotvec(np.pi/2 * np.array([0, 0, 1]))
v_imu = np.array

In [11]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def skew(v):
    """
    Create a skew-symmetric matrix from a vector.
    """
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])

def h_acc(state, a_n, omega_n):
    R_imu, v_imu, p_imu, b_a, b_omega, R_c, p_c = state

    v_c = R_imu.T @ R_c.T @ v_imu + np.cross(omega_n - b_omega, p_c)
    obs = a_n[1] - v_c.flatten()[0] * (omega_n - b_omega)[2] 
    return obs

def compute_jacobian(state, a_n, omega_n):
    # Define small perturbations
    delta = 1e-6
    n = len(state)
    jacobian = np.zeros(n)

    # Compute the observation at the nominal state
    obs_nominal = h_acc(state, a_n, omega_n)

    for i in range(n):
        # Create a perturbed state
        perturbed_state = list(state)

        if isinstance(perturbed_state[i], np.ndarray) and perturbed_state[i].shape == (3, 3):
            # Perturb a rotation matrix using the exponential map
            perturbation_vector = np.array([delta, delta, delta])
            perturbed_rotation = R.from_matrix(perturbed_state[i]) * R.from_rotvec(perturbation_vector)
            perturbed_state[i] = perturbed_rotation.as_matrix()
        elif isinstance(perturbed_state[i], np.ndarray):
            # Perturb vectors
            perturbed_state[i] += delta
        
        # Compute the observation with perturbed state
        obs_perturbed = h_acc(perturbed_state, a_n, omega_n)
        
        # Calculate Jacobian element by finite difference
        jacobian[i] = (obs_perturbed - obs_nominal) / delta

    return jacobian

Jacobian: [ 4.99933428e-07 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -4.99999000e-01 -4.99998500e-01  5.00000000e-01]


In [3]:
import numpy as np
from scipy.spatial.transform import Rotation as R


def h_acc(state, a_n, omega_n):
    R_imu, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12].reshape(3, 1), state[12:15], state[15:18], state[18:21]

    R_imu = R.from_rotvec(R_imu).as_matrix()
    R_c = R.from_rotvec(R_c).as_matrix()

    v_c = (R_c.T @ R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)
    obs = (R_c.T @ (a_n - b_a)).flatten()[1] - v_c[0] * (omega_n - b_omega)[2] 
    return obs

def compute_jacobian(x_0, a_n, omega_n, d=1e-3):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros(n)

    # Compute the observation at the nominal state
    obs_nominal = h_acc(x_0, a_n, omega_n)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc(x_0_perturbed, a_n, omega_n)
        
        # Calculate Jacobian element by finite difference
        print(obs_perturbed)
        jacobian[i] = (obs_perturbed - obs_nominal) / d

    return jacobian

# Example usage:
chi_imu = np.array([0.3, 0.2, 0.1])
v_imu = np.array([0.5, 0.0, 0.0])
p_imu = np.array([0.0, 0.0, 0.0])
b_a = np.array([0.0, 0.0, 0.0])
b_omega = np.array([0.0, 0.0, 0.0])
chi_c = np.array([0.1, 0.2, 0.3])
p_c = np.array([0.1, 0.0, 0.0])

# Nominal state

a_n = np.array([0.0, 1.0, 0.0])
omega_n = np.array([0.0, 0.0, 1.0])
x_0 = np.concatenate((chi_imu, v_imu, p_imu, b_omega, b_a, chi_c, p_c))

In [4]:
compute_jacobian(x_0, a_n, omega_n)

[0.95058062 0.95058062 0.74325797]


ValueError: setting an array element with a sequence.